<a href="https://colab.research.google.com/github/sutanmuleta/Advanced-SMS-Spam-Detection-Leveraging-BERT/blob/main/spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers


In [ ]:
from transformers import pipeline

# Load a pre-trained BERT model for text classification
classifier = pipeline("text-classification", model="bert-base-uncased")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Sample text for classification
sample_text = [
    "Free entry in 2 a weekly competition to win a prize! Text WIN to 80082.",
    "Hey, are we still on for lunch tomorrow?",
    "Congratulations, you've won a free ticket to Bahamas! Call now to claim.",
    "Your subscription to our service has been renewed. Thank you for your support."
]


In [ ]:
# Run predictions
predictions = classifier(sample_text)

# Display predictions
for text, pred in zip(sample_text, predictions):
    print(f"Text: {text}\nPrediction: {pred}\n")


Text: Free entry in 2 a weekly competition to win a prize! Text WIN to 80082.
Prediction: {'label': 'LABEL_1', 'score': 0.5322219729423523}

Text: Hey, are we still on for lunch tomorrow?
Prediction: {'label': 'LABEL_1', 'score': 0.5023919343948364}

Text: Congratulations, you've won a free ticket to Bahamas! Call now to claim.
Prediction: {'label': 'LABEL_1', 'score': 0.5292514562606812}

Text: Your subscription to our service has been renewed. Thank you for your support.
Prediction: {'label': 'LABEL_1', 'score': 0.5439599752426147}



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np


true_labels = [1, 0, 1, 0]  # Assume 1 for spam, 0 for non-spam (ham)


predicted_labels = [1 if pred['score'] > 0.5 else 0 for pred in predictions]

print(classification_report(true_labels, predicted_labels, target_names=['ham', 'spam']))


conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)


              precision    recall  f1-score   support

         ham       0.00      0.00      0.00         2
        spam       0.50      1.00      0.67         2

    accuracy                           0.50         4
   macro avg       0.25      0.50      0.33         4
weighted avg       0.25      0.50      0.33         4

Confusion Matrix:
[[0 2]
 [0 2]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Adjust the threshold to see if predictions improve
predicted_labels = [1 if pred['score'] > 0.6 else 0 for pred in predictions]  # Example threshold: 0.6

# Re-evaluate with the new threshold
print(classification_report(true_labels, predicted_labels, target_names=['ham', 'spam']))


              precision    recall  f1-score   support

         ham       0.50      1.00      0.67         2
        spam       0.00      0.00      0.00         2

    accuracy                           0.50         4
   macro avg       0.25      0.50      0.33         4
weighted avg       0.25      0.50      0.33         4



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
pip install shap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 12.8 MB/s eta 0:00:00


In [ ]:
import shap
from transformers import pipeline

# Load your trained BERT model and tokenizer
model = pipeline('text-classification', model='path_to_your_model_directory')

# Tokenize your test data
X_test_tokenized = [model.tokenizer.encode(text, truncation=True, padding='max_length', max_length=512) for text in X_test]

# Create a SHAP explainer
explainer = shap.Explainer(model, X_test_tokenized)

# Compute SHAP values
shap_values = explainer(X_test_tokenized)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: path_to_your_model_directory is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`